## 9주차 recsys basic 과제: Q1, Q2에있는 빈칸을 채워주세요

## Movielens data 탐방하기

In [3]:
!pip install src

  Using cached src-0.0.7.zip (6.3 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for src
  Running setup.py clean for src
Failed to build src
ERROR: Could not build wheels for src, which is required to install pyproject.toml-based projects


In [7]:
import argparse
# from src.preprocess.ml100k import ML100k
# from src.models.mf import MF
# from src.utils.trainer import Trainer
import torch

#다양한 hyperparameter를 조절하기 위한 argparser
argparser = argparse.ArgumentParser()
argparser.add_argument('--latent_dim', type=int, default=10)
argparser.add_argument('--batch_size', type=int, default=500)
args = argparser.parse_args("")

In [4]:
import pandas as pd
import numpy as np
# 이건 토치용입니다.
from torch.utils.data import Dataset, DataLoader
import torch

class MlDataLoader(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


## 무비렌즈 데이터 불러주는  class: ml100k
ML100k 클래스에서 data_path만 여러분이 u.data를 불러올 수 있는 경로로 바꿔주세요!!

In [8]:
import pandas as pd
# traintest split
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
#from src.preprocess.mldataloader import MlDataLoader


class ML100k():

    def __init__(self,args) -> None:
        self.args=args
        self.call_data(args)
        pass

    def call_data(self, args) -> None:
        self.args = args
        data_path="/content/u.data" #이부분을 다운받은 데이터를 코랩에서 실행할 수 잇도록 바꿔주세요

        self.data= pd.read_csv(data_path, sep='\t', header=None, names=['user', 'item', 'rating', 'timestamp'])
        self.num_users = self.data.user.nunique()
        self.num_items = self.data.item.nunique()
        self.num_ratings = len(self.data)

        # user, item index를 0부터 시작하도록 변경-. nn.Embedding에서 index가 0부터 시작하므로 1을 빼줍니다.
        self.data.user = self.data.user -1
        self.data.item = self.data.item -1


        self.x = self.data[['user', 'item']].values
        self.y= self.data.rating.values

        # split data into train and test
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.x, self.y, test_size=0.2, random_state=42)




    def get_dataframe(self):
        # sort dataframe by user
        return self.data[['user', 'item','rating']]


    def load_torch_data(self):

        # convert x to integer, y to floattenser
        self.x_train = torch.tensor(self.x_train, dtype=torch.long)
        self.y_train = torch.tensor(self.y_train, dtype=torch.float)
        self.x_test = torch.tensor(self.x_test, dtype=torch.long)
        self.y_test = torch.tensor(self.y_test, dtype=torch.float)


        train_dataset = MlDataLoader(self.x_train, self.y_train)
        test_dataset = MlDataLoader(self.x_test, self.y_test)

        train_loader = DataLoader(train_dataset, batch_size=self.args.batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=self.args.batch_size, shuffle=False)

        return train_loader, test_loader


In [9]:
ml100k=ML100k(args)
dataframe=ml100k.get_dataframe()

In [10]:
dataframe# 아래와 같이 user, item, rating으로 이루어진 데이터프레임이 나오게 됩니다.

,user,item,rating
0,195,241,3
1,185,301,3
2,21,376,1
3,243,50,2
4,165,345,1
...,...,...,...
99995,879,475,3
99996,715,203,5
99997,275,1089,1
99998,12,224,2


##  Q1 : 해당 데이터 프레임을 통해서 각각의 유저가 item들의 점수를 어떻게 매겼는지를 나타내는 user-item matrix(rating matrix)를 만들어 주세요!!
이 때 유저가 시청하지 않은 데이터는 -1로 처리해주세요!

In [14]:
# 여기에 코드를 작성해주세요
rating_matrix=dataframe.pivot_table(index='user', columns='item', values='rating', fill_value=-1)
rating_matrix

item,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
user,,,,,,,,,,,,,,,,,,,,,
0,5,3,4,3,3,5,4,1,5,3,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,4,-1,-1,-1,-1,-1,-1,-1,-1,2,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,4,3,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,-1,-1,-1,-1,-1,-1,-1,-1,5,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
939,-1,-1,-1,2,-1,-1,4,5,3,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
940,5,-1,-1,-1,-1,-1,4,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


## -1로 채워진 부분들을 채워주기 위해서 matrix factorization을 해봅시다
이때 torch를 활용해서 sgd기반의 방법으로 user embedding과 item embedding을 훈련시켜 보겠습니다.


In [15]:
# 위에 ml100k 클래스에서 torch dataloader를 불러오는 함수를 호출하여 데이터로더 형태의 train_loader, test_loader를 불러옵니다.
train_loader, test_loader=ml100k.load_torch_data()



이제 Matrix factorization model을 구현해봅시다.

In [16]:
import torch
import torch.nn as nn

class MF(nn.Module):

    def __init__(self, args ,num_users,num_items) -> None:
        super().__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.latent_dim = args.latent_dim
        self.user_embedding = nn.Embedding(self.num_users, self.latent_dim)
        self.item_embedding = nn.Embedding(self.num_items, self.latent_dim)
        #self.dropout = nn.Dropout(p=args.dropout)


    def forward(self, x) -> None:
        # x: (batch_size, 2) : index 0:user, index 1:item
        user = x[:, 0]
        item = x[:, 1]
        user = self.user_embedding(user)
        #user shape: (batch_size, latent_dim) 우리가 원하는 유저가 표현된 latent_dim 차원의 벡터
        item = self.item_embedding(item)
        #item shape: (batch_size, latent_dim) 우리가 원하는 아이템이 표현된 latent_dim 차원의 벡터

        out = torch.sum(user * item, dim=1) # 내적을 통해서 구하기,

        return out


여기서 잠깐 EMbedding이 어떻게 동작하는지 확인한번만 해봅시다

In [17]:
uemb_temp=nn.Embedding(ml100k.num_users, args.latent_dim)
iembe_temp=nn.Embedding(ml100k.num_items, args.latent_dim)

In [18]:
print(uemb_temp.weight.shape)
print(iembe_temp.weight.shape)

torch.Size([943, 10])
torch.Size([1682, 10])


In [19]:
uemb_temp.weight

Parameter containing:
tensor([[-0.1186, -0.3474,  1.5887,  ...,  0.3391,  0.8648, -0.3418],
        [-1.6642, -1.1439,  0.3435,  ...,  0.5493, -0.7319, -1.5896],
        [-0.8563,  2.0368,  0.1115,  ...,  1.2197, -1.0238,  0.8910],
        ...,
        [ 1.4463, -1.1096, -0.3453,  ...,  0.0071, -1.0671, -0.8387],
        [-0.5486, -1.0095, -0.6554,  ..., -1.1212,  2.8327, -0.8861],
        [-1.0097, -1.1979, -0.1737,  ..., -0.1683,  1.4159,  2.0215]],
       requires_grad=True)

In [ ]:
iembe_temp.weight

Parameter containing:
tensor([[ 2.6365,  0.9758, -1.1109,  ..., -0.9523, -0.8272, -0.0480],
        [-0.3241, -0.0904, -0.3940,  ..., -0.8430, -1.9926, -0.6701],
        [ 0.5003, -1.6051,  0.5715,  ..., -1.2940, -0.6821, -1.2439],
        ...,
        [-0.9983, -0.9842, -0.4111,  ..., -0.1170,  1.6426,  1.9534],
        [-0.2079, -2.2155,  1.7662,  ...,  1.3628, -0.0678, -2.6570],
        [ 0.6836, -0.6019, -1.6561,  ...,  1.5098,  1.5740,  0.8906]],
       requires_grad=True)

즉 nn.Embedding을 통해서 유저, 아이템에 대한 embedding matrix를 저장하고 훈련시킬 수 잇는 네트워크라고 보시면 됩니다.(자연어에서 사용되는 nn.embedding과 같습니다.`)

그럼 이제 모델을 훈련시켜봅시다. 엄밀히 훈련시키려면 regularization,earlystopping등의 장치가 추가적으로 필요하지만, 돌아가는거만 확인하기 위해서 최대한 간단한 형태로 구현했습니다.

In [20]:
import torch
class Trainer():

    def __init__(self, model, optimizer, criterion, device):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device


    def train(self, data, test_data, epochs):
        for epoch in range(epochs):
            for (x,y) in data:
                x = x.to(self.device)
                y = y.to(self.device)
                self.optimizer.zero_grad()
                output = self.model(x)
                loss = self.criterion(output, y)
                loss.backward()
                self.optimizer.step()
                # print test loss
            test_loss = 0
            with torch.no_grad():
                for (x,y) in test_data:
                    x = x.to(self.device)
                    y = y.to(self.device)
                    output = self.model(x)
                    test_loss += self.criterion(output, y)

                #normalize test loss
            test_loss /= len(test_data)
            print(f"Epoch {epoch} train loss: {loss.item()} test loss: {test_loss.item()}")

        print("Training finished")




In [21]:
# model, optimizer, criterion, device를 정의합니다.
model = MF(args, ml100k.num_users, ml100k.num_items)
optimizer = torch.optim.Adam(model.parameters(),lr=0.05)
criterion = nn.MSELoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion.to(device)

# trainer를 정의합니다.
trainer = Trainer(model, optimizer, criterion, device)
trainer.train(train_loader, test_loader, 10)

Epoch 0 train loss: 2.483875036239624 test loss: 2.5283918380737305
Epoch 1 train loss: 1.2354096174240112 test loss: 1.3511923551559448
Epoch 2 train loss: 0.8578453660011292 test loss: 1.2397843599319458
Epoch 3 train loss: 0.8277014493942261 test loss: 1.2219202518463135
Epoch 4 train loss: 0.9198544025421143 test loss: 1.2211912870407104
Epoch 5 train loss: 0.9059534668922424 test loss: 1.225659728050232
Epoch 6 train loss: 0.9318380355834961 test loss: 1.2348111867904663
Epoch 7 train loss: 0.8437853455543518 test loss: 1.2375792264938354
Epoch 8 train loss: 0.8976485133171082 test loss: 1.2367634773254395
Epoch 9 train loss: 0.8401255011558533 test loss: 1.2308130264282227
Training finished


이제 아까 빈칸이었던 user-item matrix를 채워봅시다!




In [35]:
user_item_matrix=dataframe.pivot(index='user', columns='item', values='rating').fillna(-1) #pandas로 하면 너무느려져서 numpy로 바구고 진행합시다
#user_item_matrix=user_item_matrix.to_numpy()

In [23]:
model=model.cpu() #모델을 cpu로 바꿔주고
# 훈련된 embedding을 numpy로 바꿔줍니다.
np_userembedding=model.user_embedding.weight.detach().numpy()
np_itemembedding=model.item_embedding.weight.detach().numpy()

## Q2. 아래 코드를 완성해주세요!

In [33]:
# 아래 코드를 완성해주세요
for i in range(ml100k.num_users):
    for j in range(ml100k.num_items):
        if user_item_matrix.iloc[i,j]==-1:
            user_item_matrix.iloc[i,j]= np_userembedding[i] @ np_itemembedding[j] # 물음표를 채워주세요! , HInt  : 내적

## 결과

In [41]:
np_userembedding[0] @ np_itemembedding[1681]

0.84982735

In [34]:
user_item_matrix.to_numpy() # user의 처음과 item의 마지막은 -1이 확실해서 해당 값의 내적을 하면 0.8x이고 -1일땐 그에 해당하는 index의 내적을 넣는 거 아닌가요?! 왜 답이 다를까요??


array([[ 5.        ,  3.        ,  4.        , ...,  0.55536699,
         2.35678506,  0.84982735],
       [ 4.        ,  3.58256936,  3.86263442, ..., -1.50003719,
         4.34184837,  4.20614386],
       [ 2.30385685,  2.40886736,  2.75307322, ...,  1.52689028,
         0.82113594, -1.03105783],
       ...,
       [ 5.        ,  3.76523685,  4.46625233, ...,  0.86099362,
         1.41124344,  4.27615118],
       [ 3.60996056,  3.08187509,  1.7817167 , ..., -1.1481353 ,
         4.5681529 ,  5.04256773],
       [ 4.16049671,  5.        ,  5.60196304, ...,  3.15359354,
         3.81716108,  3.56759381]])

In [ ]:
user_item_matrix # 정답

array([[ 5.        ,  3.        ,  4.        , ...,  3.17355108,
         1.32396555, -1.43654883],
       [ 4.        ,  3.09854269,  1.98640299, ...,  1.21267641,
        -2.27048492, -0.72931004],
       [ 2.57725286,  1.84832561,  3.45974946, ...,  4.52900362,
         2.98253703, -2.93729782],
       ...,
       [ 5.        ,  4.08410215,  3.76962662, ...,  2.65734768,
         2.2075181 , -2.00671649],
       [ 5.20034838,  3.98344159,  3.83198714, ...,  3.01264405,
         4.03274393, -2.36860776],
       [ 3.59921288,  5.        ,  4.30645037, ...,  3.45542216,
         4.21206856, -2.25273156]])

### 추가로....
결과를 보면 범위가 -3 에서  +5보다 큰 값도 나왔는데, 일반적으로는 0.5~5 사이로 normalization을 시켜주는 것 같습니다. 이건 생략하겠습니다 ㅎㅎ..